# 동의어 검색 (Synonym Search)

이 노트북에서는 Azure AI Search의 Synonym Map을 활용하여 동의어 검색을 구현합니다.

사용자가 "가방"을 검색하면 "백팩", "배낭", "크로스백" 등의 동의어도 함께 검색되도록 설정합니다.

## 0. 필요한 라이브러리 설치

In [ ]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents

## 1. 환경 변수 설정

In [ ]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")

## 2. Synonym Map 생성

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SynonymMap
import os, shutil

##############################################
# 2-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))

##############################################
# 2-1. 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    credential=credential
)

##############################################
# 2-2. 기존 Synonym Map 확인 및 삭제
##############################################
synonym_map_name = "product-synonyms"

try:
    existing_map = index_client.get_synonym_map(synonym_map_name)
    print(f"⚠️  기존 Synonym Map '{synonym_map_name}' 발견 - 삭제 중...")
    index_client.delete_synonym_map(synonym_map_name)
    print(f"✅ 기존 Synonym Map 삭제 완료")
except Exception:
    print(f"ℹ️  기존 Synonym Map 없음 - 새로 생성합니다.")


In [ ]:

##############################################
# 2-3. 동의어 정의 (Apache Solr 형식)
##############################################
synonyms = [
    "가방,백팩,배낭,숄더백",
    "운동화,스니커즈,신발,러닝화,트레이닝화",
    "여름,하계,서머",
    "겨울,동계,윈터",
    "티셔츠,티,상의,반팔티",
    "바지,팬츠,하의",
    "청바지,데님,진",
    "원피스,드레스",
    "후드티,후디,스웨트셔츠",
    "시계,손목시계,워치",
    "헤드폰,이어폰,음향기기",
    "셔츠,남방,블라우스",
    "부츠,앵클부츠,워커",
    "선글라스,썬글라스,안경",
    "조거팬츠,트레이닝복",
    "스마트워치,웨어러블",
    "카디건,가디건,겉옷",
    "와이드팬츠,통바지,슬랙스",
    "패딩,다운,점퍼,파카,아우터",
    "스카프,머플러,목도리",
    "레깅스,스포츠바지,요가복"
]


##############################################
# 2-4. Synonym Map 생성
##############################################
synonym_map = SynonymMap(
    name=synonym_map_name,
    format="solr",
    synonyms=synonyms
)

try:
    index_client.create_or_update_synonym_map(synonym_map)
    print(f"✅ Synonym Map '{synonym_map_name}' 생성 완료")
except Exception as e:
    print(f"❌ Synonym Map 생성 실패: {e}")

##############################################
# 2-5. 생성된 Synonym Map 확인
##############################################
try:
    created_map = index_client.get_synonym_map(synonym_map_name)
    # synonyms가 리스트일 수도, 문자열일 수도 있으므로 처리
    if isinstance(created_map.synonyms, list):
        synonym_count = len(created_map.synonyms)
    else:
        synonym_count = len([line for line in created_map.synonyms.split('\n') if line.strip()])
    print(f"✅ Synonym Map 확인 완료 - 총 {synonym_count}개의 동의어 그룹 등록됨")
except Exception as e:
    print(f"❌ Synonym Map 확인 실패: {e}")


## 3. 인덱스에 Synonym Map 적용

In [ ]:
##############################################
# 3-1. Synonym Map 적용 전 - 기존 설정 제거
##############################################
index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

print("📋 Synonym Map 제거 전 상태:")
for field in index.fields:
    if field.name in ["name", "description"]:
        if field.synonym_map_names:
            print(f"   - {field.name}: {field.synonym_map_names}")
        else:
            print(f"   - {field.name}: 없음")

# 기존 Synonym Map 제거
print("\n⚠️  기존 Synonym Map 제거 중...")
for field in index.fields:
    if field.name in ["name", "description"]:
        field.synonym_map_names = []

try:
    index_client.create_or_update_index(index)
    print("✅ 기존 Synonym Map 제거 완료")
except Exception as e:
    print(f"❌ Synonym Map 제거 실패: {e}")

##############################################
# 3-2. 제거 확인
##############################################
index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

print("\n📋 제거 후 상태:")
for field in index.fields:
    if field.name in ["name", "description"]:
        if field.synonym_map_names:
            print(f"   ⚠️  {field.name}: {field.synonym_map_names} (아직 남아있음)")
        else:
            print(f"   ✅ {field.name}: 없음 (제거됨)")

##############################################
# 3-3. 새로운 Synonym Map 적용
##############################################
print("\n🔧 새로운 Synonym Map 적용 중...")
for field in index.fields:
    if field.name in ["name", "description"]:
        field.synonym_map_names = ["product-synonyms"]

try:
    index_client.create_or_update_index(index)
    print("✅ Synonym Map 적용 완료")
except Exception as e:
    print(f"❌ Synonym Map 적용 실패: {e}")

##############################################
# 3-4. 적용 확인
##############################################
final_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

print("\n📋 최종 적용 상태:")
for field in final_index.fields:
    if field.name in ["name", "description"]:
        if field.synonym_map_names:
            print(f"   ✅ {field.name}: {field.synonym_map_names}")
        else:
            print(f"   ❌ {field.name}: 없음 (적용 실패)")

print("\n" + "="*60)
print("✅ Synonym Map 적용 프로세스 완료")
print("="*60)

## 4. Synonym Map 검색 테스트

In [ ]:
from azure.search.documents import SearchClient
from IPython.display import display, HTML

##############################################
# 4-1. 검색 클라이언트 생성
##############################################
search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

##############################################
# 4-2. 동의어 검색 테스트
##############################################
search_query = "가방"

results = search_client.search(
    search_text=search_query,
    top=5,
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"\n{'='*60}")
print(f"검색어: '{search_query}'")
print(f"{'='*60}")

result_list = list(results)
if result_list:
    for idx, result in enumerate(result_list, 1):
        print(f"\n{idx}. {result['name']} ({result['brand']})")
        print(f"   가격: {result['price']:,.0f}원")
        print(f"   설명: {result['description']}")
        display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
else:
    print("검색 결과 없음")


## 5. Synonym Map 제거

In [ ]:
##############################################
# 5. Synonym Map 제거
##############################################

DELETE_SYNONYM_MAP = False

if DELETE_SYNONYM_MAP:
    # 5-1. 인덱스 필드에서 Synonym Map 제거
    index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

    print("\n⚠️ 인덱스 필드에서 Synonym Map 제거 중...")
    for field in index.fields:
        if field.name in ["name", "description"]:
            field.synonym_map_names = []

    try:
        index_client.create_or_update_index(index)
        print("✅ 인덱스에서 Synonym Map 제거 완료")
    except Exception as e:
        print(f"❌ 인덱스 제거 실패: {e}")

    # 5-2. Synonym Map 자체 삭제
    print("\n⚠️ Synonym Map 'product-synonyms' 삭제 중...")
    try:
        index_client.delete_synonym_map("product-synonyms")
        print("✅ Synonym Map 삭제 완료")
    except Exception as e:
        print(f"❌ Synonym Map 삭제 실패: {e}")

    # 5-3. 제거 확인
    updated_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

    print("\n📋 제거 후 상태:")
    for field in updated_index.fields:
        if field.name in ["name", "description"]:
            if field.synonym_map_names:
                print(f"   ⚠️ {field.name}: {field.synonym_map_names}")
            else:
                print(f"   ✅ {field.name}: 없음")

    try:
        check_map = index_client.get_synonym_map("product-synonyms")
        print(f"\n⚠️ 경고: Synonym Map 'product-synonyms'가 여전히 존재합니다!")
    except Exception:
        print(f"\n✅ Synonym Map 'product-synonyms' 완전히 제거됨")

    print("\n" + "="*60)
    print("✅ Synonym Map 제거 프로세스 완료")
    print("="*60)